In [ ]:
# 1.canny边缘检测  2.mask   3.霍夫变换   4.离群值过滤    5.最小二乘拟合     6.绘制直线
import math
import re
from traceback import print_tb
import cv2
import numpy as np
import matplotlib.pyplot as plt
from regex import T
from skimage import morphology
from sympy import true

filepath='frame/154.jpg'
img0 = cv2.imread(filepath, cv2.IMREAD_COLOR)
frame=cv2.imread(filepath)

def edge(frame):
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY) #转换为灰度图
    blurred = cv2.GaussianBlur(gray,(5,5),0) #高斯滤波
    edge = cv2.Canny(blurred,50,150) #canny边缘检测
    return edge
def hsv(frame):
    dst = cv2.GaussianBlur(frame,(5,5),0) #对图像进行第一次高斯滤波
    # dst = cv2.medianBlur(frame,5)
    hsv = cv2.cvtColor(dst,cv2.COLOR_BGR2HSV) #使用HSV颜色空间
    lower_blue = np.array([81,79,77])#设置需要识别的车道线颜色的阈值
    upper_blue = np.array([161,255,185])
    lower_yellow= np.array([15,43,46])
    upper_yellow = np.array([40,255,255])
    lower_white = np.array([0,0,200])
    upper_white = np.array([180,30,255])
    # dark_blue = np.uint8([[[12,22,121]]])
    # dark_blue = cv2.cvtColor(dark_blue,cv2.COLOR_BGR2HSV)

    mask = cv2.inRange(hsv,lower_yellow,upper_yellow) + cv2.inRange(hsv,lower_white,upper_white) #提取所需颜色
    # 先腐蚀后膨胀
    # mask=cv2.inRange(hsv,lower_blue,upper_blue)
    mask = morphology.erosion(mask,morphology.square(width=5)) #形态学方法滤除噪声
    mask= morphology.dilation(mask,morphology.square(11))
    mask= morphology.erosion(mask,morphology.square(5))
    return mask


def calculate_slope(line):
    if len(line[0]) == 4:
        x_1, y_1, x_2, y_2 = line[0]
    else:
        x_1 = line[0][0]
        y_1 = line[0][1]
        x_2 = line[1][0]
        y_2 = line[1][1]
    return (y_2 - y_1) / (x_2 - x_1)


hsv_mask=edge(frame)
cv2.imshow('hsv_mask',hsv_mask)
cv2.waitKey(0)
"""2.roi_mask(提取感兴趣的区域)"""
mask = np.zeros_like(hsv_mask)  # 变换为numpy格式的图片
print(mask.shape)
mask = cv2.fillPoly(
    mask,
    np.array(
        object=[
            [
                [0, 250],
                [950, 250],
                [950, 544],
                [0, 544],
            ]
        ]
    ),
    color=255,
)  # 对感兴趣区域制作掩膜
cv2.imshow('mask',mask)
cv2.waitKey(0)
masked_edge_img = cv2.bitwise_and(hsv_mask, mask)  # 与运算
cv2.imshow('masked_edge_img',masked_edge_img)
cv2.waitKey(0)


# 连通域分析，去除小面积的连通域
num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(masked_edge_img, connectivity=8)
filtered = np.zeros_like(masked_edge_img)
for i in range(1, num_labels):  # 跳过背景
    area = stats[i, cv2.CC_STAT_AREA]
    print(area)
    if area >200:  # 设置面积阈值，保留较大的线段（可调整）
        filtered[labels == i] = 255
img0[filtered>0]=[0,0,255]
masked_edge_img=cv2.bitwise_and(filtered,masked_edge_img)

#
lines = cv2.HoughLinesP(
    masked_edge_img, 1, np.pi / 180, 15, minLineLength=50, maxLineGap=10
)  # 获取所有线段

left_lines = [line for line in lines if calculate_slope(line) > 0]
right_lines = [line for line in lines if calculate_slope(line) < 0]


"""4.离群值过滤"""


def reject_abnormal_lines(lines, threshold):
    """剔出斜率不一致的线段"""
    slopes = [calculate_slope(line) for line in lines]
    while len(lines) > 0:
        mean = np.mean(slopes)
        diff = [abs(s - mean) for s in slopes]
        idx = np.argmax(diff)
        if diff[idx] > threshold:
            slopes.pop(idx)
            lines.pop(idx)
        else:
            break
    return lines


reject_abnormal_lines(left_lines, threshold=0.1)
reject_abnormal_lines(right_lines, threshold=0.1)

"""5.最小二乘拟合 把识别到的多条线段拟合成一条直线"""
# np.ravel: 将高维数组拉成一维数组
# np.polyfit:多项式拟合
# np.polyval: 多项式求值


def least_squares_fit(lines):

    x_coords = np.ravel([[line[0][0], line[0][2]] for line in lines])
    y_coords = np.ravel([[line[0][1], line[0][3]] for line in lines])  # 取出所有标点
    poly = np.polyfit(x_coords, y_coords, deg=1)  # 进行直线拟合，得到多项式系数
    point_min = (np.min(x_coords), np.polyval(poly, np.min(x_coords)))
    point_max = (
        np.max(x_coords),
        np.polyval(poly, np.max(x_coords)),
    )  # 根据多项式系数，计算两个直线上的点
    return np.array([point_min, point_max], dtype=np.int64)

if len(left_lines)!=0:
    left_lines = least_squares_fit(left_lines)
else:
    left_lines=[[0,0],[0,0]]
if len(right_lines)!=0:
    right_lines = least_squares_fit(right_lines)
else:
    right_lines=[[0,0],[0,0]]

left_length=math.sqrt((left_lines[0][0]-left_lines[1][0])**2+(left_lines[0][1]-left_lines[1][1])**2)
right_length=math.sqrt((right_lines[0][0]-right_lines[1][0])**2+(right_lines[0][1]-right_lines[1][1])**2)

def is_vertical(left_lines, right_lines):
    """剔除垂直的线段"""
    k1=calculate_slope(left_lines)
    k2=calculate_slope(right_lines)
    theta1=math.atan(k1)
    theta2=math.atan(k2)
    print(theta1,theta2)
    delta=math.pi+theta2-theta1
    max_delta=math.pi*1/3
    if delta>max_delta:
       return True
    else:
        return False

if is_vertical(left_lines, right_lines):
    if left_length<right_length:
        left_lines=[[0,0],[0,0]]
    else:
        right_lines=[[0,0],[0,0]] 

"""6.直线绘制"""
cv2.line(
    img0, tuple(left_lines[0]), tuple(left_lines[1]), color=(0, 255, 255), thickness=5
)
cv2.line(
    img0, tuple(right_lines[0]), tuple(right_lines[1]), color=(0, 255, 255), thickness=5
)

cv2.namedWindow("lane", 0)
cv2.resizeWindow("lane", 800, 1200)
cv2.imshow("lane", img0)
cv2.waitKey(0)

(544, 960)
82
731
223
740
36
0.34555558058171215 -0.22690423323633901


-1